# SIT742: Modern Data Science 
**(Module 04: Exploratory Data Analysis)**


---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip-lab/sit742](https://github.com/tulip-lab/sit742/issues)

Prepared by **SIT742 Teaching Team**

---



# Session 4D - Association Rule Mining

Apiori Algorithm is a Association Rule learning Algorithm and is used to find frequent itemsets and relevant association rules. By frequent itemsets we should think of items who are frequently associated with each other within the whole dataset. In order for a frequent itemset to be valid the subsets of the itemset needs to be frequent by themselves, this is called anti-monotonicity or downward-closure poperty. The anti-monotonicity guarantees a efficient search, this is necessary because the algorithm goes through all combinations adding one at a time. The process of adding one at a time is called buttom up approach.

## `apyori` Package

In this part, we will use the `apyori` package. 

Here we assume a simple transaction set, and then the calling of `apriori` is straightforward:

In [ ]:
!pip install apriori

In [ ]:
!pip install apyori

In [ ]:
from apyori import apriori

transactions = [
    ['beer', 'nuts'],
    ['beer', 'cheese'],
]

rules = apriori(transactions)

# rules = apriori(transactions, min_support = 0.5, min_confidence = 0.7, min_length = 2)


results = list(rules)

In [ ]:
print(results)

In [ ]:
for i in range(len(results)):
    print("##############################################################################")
    print(i)
    print(results[i])
    print(results[i].items)

## Transactions from File

If the transaction set is stored on an external file, we might need to use some code to load it into the proper format:

Consider a file with the following content:

![DS1](http://franklinwillemen.com/machine-learning/assets/img/ML/Association_Rule_Learning/apriori_dataset1.jpg "Data Structure")

When importing our dataset `Market_Basket_Optimization`, we should add the `header` argument with the value `None`, without this Python places the first row of the dataset as the titles of the columns and we do not want this. If we take a look at the dataframe now we can see that our product names are now located in the first row. 


The next thing is that we will preprocess our data set by placing it in Lists of Lists, we do this using two for loops. The first for loop goes through all the rows (transactions) of the dataset and the second for loop goes through all the products and adds these to a list before appending them to the transactions list. We can see that our dataset is now of the type list and that it's value is indeed a list of products. 

![DS2](http://franklinwillemen.com/machine-learning/assets/img/ML/Association_Rule_Learning/apriori_dataset2.jpg "Data Structure2")


We will not go into details of this code because there are various ways to get to the same result with Python, just note that the second for loop is actually a inner for loop, nested in the append method. 

Also note that the `dataset.values[i,j]` is converted to a string value when appended to the transaction list.

In [ ]:
!pip install wget

In [ ]:
import wget

link_to_data = 'https://github.com/tuliplab/mds/raw/master/Jupyter/data/Market_Basket_Optimisation.csv'
DataSet = wget.download(link_to_data)

In [ ]:
import pandas as pd

# Data Preprocessing
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)
transactions = []
for i in range(0, 7501):
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

Before we train our apriori algorithm on the dataset, let's import the apyori.py library which contains the classes we will be using. We will declare a variable called rules since the apriori algorithm takes the transactions list as input and outputs association rules. If we inspect our Apiori function we see it accepts one main argument and that is an iterable object our transactions list. We can also specify the support, confidence and lift thresholds. A new argument which we have not discussed yet is min_lenght, it specify's the minimum length of our rules, we will set this to 2 to prevent rules with only one item.

When deciding on the minimum support we should calculate this in a way that it makes sense according to the problem at hand. We want to optimize the sales for a supermarket, the dataset contains all the transactions made in one week. If we want to have products who are minimally bought 3 times a day our minimum support will be 0.0028. (3*7/7500)

The minimum confidence should prevent that rules are made based on the wrong reasons, there are products who are bought frequently together not because they're strongly associated but because they are frequently bought products independent of each other. We want our rules to be relevant, choosing 0.8 is to high and will give us no results on this dataset, 0.4 will give us irrelevant rules as described earlier, the winner is 0.2 because this confidence minimum gave us relevant rules that were proven true enough times but not too frequent to be irrelevant to our goal. This minimum is found by running the Apriori algorithm different times and checking if the results makes sense. 

The lift measurement is a good indication of the relevance of our rules, we will set our minimum lift to 3 because we can be sure that these rules will be significant. Remember that these arguments depends on your dataset or problem. The Apriori is an experimental algorithm, we need to try different threshold and see if the produced rules have the wanted effect in the real world if not than we have to tune our thresholds some more.

In [ ]:
# Training Apriori on the dataset
from apyori import apriori
rules = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

# Visualising the results
results = list(rules)

myResults = [list(x) for x in results] 


In [ ]:
print(myResults)

In [ ]:
for i in range(len(results)):
    print("##############################################################################")
    print(i)
    print(results[i])
    print(results[i].items)